In [ ]:
import pandas as pd
import tensorflow as tf
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Flatten,Dense,MaxPooling2D,Conv2D,BatchNormalization,Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import TensorBoard
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
file_format = ('.jpg','.jpeg','.png')

def corrupt_check(dir):
  count=0
  for root,_,files in os.walk(dir):
      for file in files:
          if file.endswith(file_format):
              try:
                img=Image.open(os.path.join(root,file))
                img.verify()
                #print(f"file path{os.path.join(root,file)}")
                count +=1
              except:
                print(f"corrrupt file path{os.path.join(root,file)}")
                #os.remove(os.path.join(root,file))
          else:
              print(f"unsupported file {os.path.join(root,file)}")
  print(count)

file_path_train='G:/aman office/practise/face_emotion_detection/train'
corrupt_check(file_path_train)

In [ ]:
file_format = ('.jpg','.jpeg','.png')

def corrupt_check(dir):
  count=0
  for root,_,files in os.walk(dir):
      for file in files:
          if file.endswith(file_format):
              try:
                img=Image.open(os.path.join(root,file))
                img.verify()
                #print(f"file path{os.path.join(root,file)}")
                count +=1
              except:
                print(f"corrrupt file path{os.path.join(root,file)}")
                #os.remove(os.path.join(root,file))
          else:
              print(f"unsupported file {os.path.join(root,file)}")
  print(count)

file_path_test='G:/aman office/practise/face_emotion_detection/test'
corrupt_check(file_path_test)

In [ ]:
train_dir = file_path_train
validation_dir = file_path_test

In [ ]:
# Image Data Generator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)
validation_datagen = ImageDataGenerator(rescale=1./255)



In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(48, 48),
    batch_size=64,
    class_mode='categorical',
    color_mode='grayscale'  
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(48, 48),
    batch_size=64,
    class_mode='categorical',
    color_mode='grayscale' 
)

In [ ]:
# 3. CNN Model 
model = Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=(48, 48, 1)),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    Dropout(0.25),

    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    Dropout(0.25),

    Conv2D(256, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    Dropout(0.25),

    Flatten(),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),

    Dense(7, activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
class_names = list(train_generator.class_indices.keys())
print(class_names)


In [ ]:
num_classes = len(train_generator.class_indices) 
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(48, 48, 1)),  
    MaxPooling2D(2,2),
    BatchNormalization(),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    BatchNormalization(),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    BatchNormalization(),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

In [ ]:
# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6, verbose=1)

# Model Training
history = model.fit(
    train_generator,
    epochs=50,
    validation_data=validation_generator,
    callbacks=[early_stopping, lr_scheduler]
)


In [ ]:
model.save("best_model.h5")


In [ ]:
from tensorflow.keras.models import load_model
model = load_model('G:/aman office/practise/face_emotion_detection/face_emotion_detection.h5')

val_images, val_labels = next(validation_generator)
predictions = model.predict(val_images)
y_true = np.argmax(val_labels, axis=1)
y_pred = np.argmax(predictions, axis=1)

print(classification_report(y_true, y_pred, target_names=class_names))